In [1]:
import sys
from os.path import join as pj

import sc
from sc.lib import *
import os
import argparse
import logging
import os.path
import time

In [2]:
def get_artifact_path(filename):
    packagedir = sc.__path__[0]
    dirname = pj(os.path.dirname(packagedir), 'share', 'data')
    fullname = os.path.join(dirname, filename)
    return fullname


def get_all_analysis_classes():
    config_file = pkg_resources.resource_stream("sc", 'share/data/artifacts/analysis_classes.csv')
    utf8_reader = codecs.getreader("utf-8")
    c = csv.DictReader(utf8_reader(config_file))
    l = []
    for record in c:
        l.append(record['classes'])
    return l


def choose_classes(acs, data_group_by_class):
    acs = list(map(lambda x: x.replace(".", "_"), acs))
    rm_keys = []
    for dki in data_group_by_class:
        if len(data_group_by_class[dki]) < 30:
            rm_keys.append(dki)
        if dki not in acs:
            rm_keys.append(dki)
    for rk in rm_keys:
        if rk in data_group_by_class:
            del data_group_by_class[rk]
    return data_group_by_class


def dir_check(f):
    if not os.path.isdir(f):
        logger.error("%s is not dir" % f)
        exit(1)

In [5]:
f1 = "./experimental_data/data_folder"

In [6]:
analysis_classes = get_all_analysis_classes()

In [7]:
o1 = get_data_group(f1)

In [8]:
f_budget = "./experimental_data/budget"
if f_budget is not None and f_budget != "":
    dir_check(f_budget)
    o1 = get_data_group(f_budget, o1)

In [9]:
f1_data_group_by_class = choose_classes(analysis_classes, o1)

In [10]:
cns = []
for k,v in f1_data_group_by_class.items():
    class_name = v['con-cbranch-dynamosa-1.2.0']['data'][v['con-cbranch-dynamosa-1.2.0']['classes'][0][1]]['TARGET_CLASS'][0]
    cns.append(class_name)
    print(v.keys())

dict_keys(['con-cbranch-dynamosa-1.2.0', 'con-exce-dynamosa-1.2.0', 'con-output-dynamosa-1.2.0', 'con-methodne-dynamosa-1.2.0', 'con-line-suite-1.2.0', 'con-wm-suite-1.2.0', 'con-branch-suite-1.2.0', 'con-method-suite-1.2.0', 'origin-suite-1.2.0', 'origin-mosa-1.2.0', 'origin-dynamosa-1.2.0', 'sc-dynamosa-sc-release1', 'con-branch-dynamosa-1.2.0', 'con-wm-mosa-1.2.0', 'con-exce-mosa-1.2.0', 'con-output-mosa-1.2.0', 'con-branch-mosa-1.2.0', 'con-wm-dynamosa-1.2.0', 'con-line-dynamosa-1.2.0', 'con-method-dynamosa-1.2.0', 'con-cbranch-suite-1.2.0', 'con-output-suite-1.2.0', 'con-exce-suite-1.2.0', 'con-methodne-suite-1.2.0', 'sc-suite-sc-release1', 'sc-mosa-sc-release1', 'con-cbranch-mosa-1.2.0', 'con-line-mosa-1.2.0', 'con-method-mosa-1.2.0', 'con-methodne-mosa-1.2.0', 'con-cbranch-5-mosa-1.2.0', 'con-exce-5-mosa-1.2.0', 'con-methodne-8-mosa-1.2.0', 'con-output-8-mosa-1.2.0', 'con-exce-8-mosa-1.2.0', 'con-cbranch-8-mosa-1.2.0', 'con-output-5-mosa-1.2.0', 'con-methodne-5-mosa-1.2.0', 'con

In [11]:
cns

['org.apache.hadoop.security.authentication.util.ZKSignerSecretProvider',
 'accessories.plugins.time.JDayChooser']

In [13]:
ad_suite = analysis_data_4_alg(f1_data_group_by_class, 'suite')

In [22]:
def constituent_map_4_budget(algorithm, budget):
    return {
        ("con-branch-%d-%s-1.2.0" % (budget, algorithm)): "BranchCoverageBitString",
        ("con-wm-%d-%s-1.2.0" % (budget, algorithm)): "WeakMutationCoverageBitString",
        ("con-line-%d-%s-1.2.0" % (budget, algorithm)): "LineCoverageBitString",
        ("con-method-%d-%s-1.2.0" % (budget, algorithm)): "MethodCoverageBitString",
        ("con-methodne-%d-%s-1.2.0" % (budget, algorithm)): "MethodNoExceptionCoverageBitString",
        ("con-cbranch-%d-%s-1.2.0" % (budget, algorithm)): "CBranchCoverageBitString",
        ("con-exce-%d-%s-1.2.0" % (budget, algorithm)): "ExceptionCoverageBitString",
        ("con-output-%d-%s-1.2.0" % (budget, algorithm)): "OutputCoverageBitString"
    }
def concat_constituent_one_class_4_budget(algorithm, data_group_by_class, a_class, budget):
    m = constituent_map_4_budget(algorithm, budget)
    g = []
    i = 0
    for k, v in m.items():
        if i == 0:
            g.append(data_group_by_class[a_class][k]["data"][a_class])
        else:
            g.append(data_group_by_class[a_class][k]["data"][a_class][v])
        new_value = v.replace('BitString', '')
        size_slice = data_group_by_class[a_class][k]["data"][a_class]["Size"].copy()
        size_slice = size_slice.rename("Constituent" + new_value + "Size")
        g.append(size_slice)
        i = i + 1
    return pd.concat(g, axis=1)
def concat_constituent_all_4_budget(algorithm, data_group_by_class, budget=5):
    for ac in list(data_group_by_class.keys()):
        is_full = True
        ks = constituent_map_4_budget(algorithm, budget)
        for k, _ in ks.items():
            if not k in data_group_by_class[ac]:
                is_full = False
                break
        if not is_full:
            continue
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)] = {}
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["classes"] = \
            data_group_by_class[ac]["origin-%s-1.2.0" % (algorithm)]["classes"]
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["data"] = {}
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["data"][ac] = \
            concat_constituent_one_class_4_budget(algorithm, data_group_by_class, ac, budget)
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["name"] = "constituent-%d-%s" % (budget, algorithm)
    return data_group_by_class

In [28]:
f1_data_group_by_class = concat_constituent_all_4_budget('suite', f1_data_group_by_class, 10)

In [30]:
f1_data_group_by_class['org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider']['constituent-10-suite']['data']['org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider']

,TARGET_CLASS,search_budget,Size,Length,Generations,BranchCoverageBitString,ConstituentBranchCoverageSize,WeakMutationCoverageBitString,ConstituentWeakMutationCoverageSize,LineCoverageBitString,...,MethodCoverageBitString,ConstituentMethodCoverageSize,MethodNoExceptionCoverageBitString,ConstituentMethodNoExceptionCoverageSize,CBranchCoverageBitString,ConstituentCBranchCoverageSize,ExceptionCoverageBitString,ConstituentExceptionCoverageSize,OutputCoverageBitString,ConstituentOutputCoverageSize
0,org.apache.hadoop.security.authentication.util...,600,8,135,20,1111110010000100001000011100000010000000000000...,8,1110111111111111110000000000001101110011111111...,12,1111111111000000111111111101111111111111111111...,...,111111111000,6,111111010000,7,1011110100011000001000100100000010000000000000...,4,1111,6,1100001010,14
1,org.apache.hadoop.security.authentication.util...,600,10,144,29,1111110010000100001000010100000010000000000000...,10,1110111111111111110000000000001101110011111111...,3,1111111111000000111111111101111111111111111111...,...,111111111000,6,111111010000,4,1011110100011000001000101100000010000000000000...,4,1111,10,1100001010,13
2,org.apache.hadoop.security.authentication.util...,600,3,16,23,1111110010000100001000010100000010000000000000...,3,1110111111111111110000000000001101110011111111...,3,1111111111000000111111111101111111111111111111...,...,111111111000,8,111111010000,5,1011110100011000001000100100000010000000000000...,11,1111,14,1100001010,8
3,org.apache.hadoop.security.authentication.util...,600,9,160,17,1111110010000100001000010100000010000000000000...,9,1110111111111111110000000000001101110011111111...,8,1111111111000000111111111101111111111111111111...,...,111111111000,7,111111010000,4,1011110100011000001000101100000010000000000000...,9,1111,10,1100001010,14
4,org.apache.hadoop.security.authentication.util...,600,11,208,24,1111110010000100001000111100000010000000000000...,11,1110111111111111110000000000001101110011111111...,15,1111111111000000111111111101111111111111111111...,...,111111111000,8,111111010000,7,1011110100011000001000100100000010000000000000...,3,1111,5,1100001010,10
5,org.apache.hadoop.security.authentication.util...,600,12,123,22,1111110010000100001000111100000010000000000000...,12,1110111111111111110000000000001101110011111111...,5,1111111111000000111111111101111111111111111111...,...,111111111000,7,111111010000,6,1011110100011000001000000000000010000000000000...,4,1111,17,1100001010,12
